In [28]:
import pandas as pd
from pyhive import hive
from matplotlib import pyplot as plt
host='*****'
username='*****'
password='*****'
port=10000
data_base_name='*****'

#连接hive
conn = hive.Connection(host=host,
                       port=port,
                       auth="CUSTOM",
                       database=data_base_name,
                       username=username,
                       password=password)
cur = conn.cursor()

#读取数据
sql_order ='select * from dw_sd_park_forecast_stat_info where dw_sd_park_forecast_stat_info.city = "杭州市"and dw_sd_park_forecast_stat_info.dt = "2022-06-29"'
forecast_data = pd.read_sql(sql_order, conn)
forecast_data.rename(columns={"7_day_order_count":"week_day_order_count"},inplace=True)
#模型推理
import joblib
model_rf = joblib.load('C:/Users/Liangliang.Xu1.GEELY/IdeaProjects/final/loan_model_rf.pkl')
forecast_feature = ['dc_count', 'equipment_count', 'three_order_3km', 'total_order_3km', 'dc_count_3km', 'd_7day_total_order_count', 'd_7day_three_order_count', 'xj_order_3km', 'operate_construction', 'd_total_dc_count', 'd_7day_tld_order_count', 'd_7day_xj_order_count', 'd_7day_xx_order_count', 'd_house_count', 'equipment_count_3km', 'c_xp_total_count', 'd_park_count', 'd_populace_density', 'd_shopping_count', 'station_total_3km', 'd_work_count']
feature_data_fore = forecast_data[forecast_feature]
feature_data_fore.isnull().sum()
feature_data_fore['operate_construction'].fillna(0, inplace=True)
feature_data_fore.isnull().sum()
rf_fore = model_rf.predict(feature_data_fore)
forecast_data = forecast_data[["park_id", "park_name", "city", "city_code", "district", "district_code", "location","address"]+forecast_feature]
forecast_data["rf_fore"] = rf_fore
forecast_data.to_csv("C:/Users/IdeaProjects/final/park_hangzhou.csv",encoding = "utf_8_sig")
print(forecast_data.head())

C:\Users\Liangliang.Xu1.GEELY\Anaconda3\envs\location\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


      park_id      park_name city city_code district district_code  \
0  B023B07KM1  杭州野生动物世界A区停车场  杭州市    330100      富阳区        330111   
1  B023B07ULB     红灯笼乡村家园停车场  杭州市    330100      桐庐县        330122   
2  B023B08VRK       建德服务区停车场  杭州市    330100      建德市        330182   
3  B023B08WUY          彩园停车场  杭州市    330100      上城区        330102   
4  B023B097S8     花筑·白云山庄停车场  杭州市    330100      淳安县        330127   

               location            address  dc_count  equipment_count  ...  \
0  119.987354,30.150732  受降镇九龙大道1号杭州野生动物世界        10               10  ...   
1  119.520423,29.913239                白冷线        10               10  ...   
2  119.387432,29.543236   乌驹市村建德服务北区1号商铺附近        10               10  ...   
3  120.219172,30.254626       庆和路与钱潮路交汇处彩园        10               10  ...   
4  119.007953,29.373555       淳安石林镇石林景区停车场        10               10  ...   

   d_7day_xx_order_count  d_house_count  equipment_count_3km  \
0                    406            494       